In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint as pp

from qiskit import *
from qiskit import qobj as qiskit_qobj
from qiskit.providers import BaseBackend, BaseJob, BaseProvider
from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator
from qiskit.providers.models import BackendConfiguration
from qiskit.providers.models.backendconfiguration import GateConfig
from qiskit.providers.providerutils import filter_backends
from qiskit.transpiler import PassManager
from qiskit.result import Result
from qiskit.tools.monitor import job_monitor
from qiskit.transpiler import PassManager
from qiskit.visualization import plot_histogram, plot_gate_map, plot_circuit_layout, plot_state_city


In [2]:
tmp=IBMQ.load_account().get_backend('ibmq_16_melbourne').configuration()

tmp.gates


[GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13]], name='id', parameters=[], qasm_def='gate id q { U(0,0,0) q; }'),
 GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13]], name='u1', parameters=['lambda'], qasm_def='gate u1(lambda) q { U(0,0,lambda) q; }'),
 GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13]], name='u2', parameters=['phi', 'lambda'], qasm_def='gate u2(phi,lambda) q { U(pi/2,phi,lambda) q; }'),
 GateConfig(coupling_map=[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13]], name='u3', parameters=['theta', 'phi', 'lambda'], qasm_def='u3(theta,phi,lambda) q { U(theta,phi,lambda) q; }'),
 GateConfig(coupling_map=[[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]], name='cx', parameters=[], qasm_def='gate cx q1,q

In [3]:
class OpenSuperQ_Job(BaseJob):
    def __init__(self, backend):
        super().__init__(backend, 1)
        
    def result(self):
        return self._result
    
    def cancel(self):
        pass
    
    def status(self):
        pass
    
    def submit(self):
        pass
    
class ETH_7_rev_1_Backend(BaseBackend):
    '''
    A wrapper backend for the ETH 7 rev 1 chip
    '''

    def __init__(self, provider=None):
        configuration = {
            'backend_name':     'OSQ_ETH7_rev1',
            'description':      'OpenSuperQ ETH 7 qubit chip, rev. 1, 12_15702', 
            'backend_version':  '0.1.0',
            'url':              'http://opensuperq.eu/',
            'sample_name':      'QUDEV_M_12_15702', 
            'n_qubits':         7, 
            'basis_gates':      ['u1', 'u2', 'u3', 'cx', 'id'], 
            'coupling_map':     [[0,2],[0,3], [1,3],[1,4], [2,5], [3,5],[3,6], [4,6]], # Reduced qubit numbers by 1 compared to 20190823_OSQ_Waypoint1_experimental_parameters.pdf
            'simulator':        True, 
            'local':            True, 
            'open_pulse':       False, 
            'conditional':      False, 
            'n_registers':      1,   # Really 0, but QISKIT would not allow it, even if 'conditional' is False
            'max_shots':        1_000_000,
            'memory':           0,
            'credits_required': False, 
            'gates': [
                GateConfig(
                    name         = 'id', 
                    parameters   = [], 
                    coupling_map = [[0], [1], [2], [3], [4], [5], [6]], 
                    qasm_def     = 'gate id q { U(0,0,0) q; }'), 
                GateConfig(
                    name         = 'u1', 
                    parameters   = ['lambda'], 
                    coupling_map = [[0], [1], [2], [3], [4], [5], [6]], 
                    qasm_def     = 'gate u1(lambda) q { U(0,0,lambda) q; }'), 
                GateConfig(
                    name         = 'u2',
                    parameters   = ['phi', 'lambda'],  
                    coupling_map = [[0], [1], [2], [3], [4], [5], [6]], 
                    qasm_def     = 'gate u2(phi,lambda) q { U(pi/2,phi,lambda) q; }'), 
                GateConfig(
                    name         = 'u3',
                    parameters   = ['theta', 'phi', 'lambda'], 
                    coupling_map = [[0], [1], [2], [3], [4], [5], [6]],
                    qasm_def     = 'u3(theta,phi,lambda) q { U(theta,phi,lambda) q; }'), 
                GateConfig(
                    name         = 'cx',
                    parameters   = [],
                    coupling_map = [[0,2],[0,3], [1,3],[1,4], [2,5], [3,5],[3,6], [4,6]],
                    qasm_def     = 'gate cx q1,q2 { CX q1,q2; }')
            ]
        }
                
       
        super().__init__(configuration=BackendConfiguration.from_dict(configuration),
                         provider=provider)
        
        self.aer_simulator = Aer.get_backend('qasm_simulator')


    def run(self, qobj):
        """Run qobj

        Args:
            qobj (QObj): circuit description

        Returns:
            OpenSuperQ_Job: derived from BaseJob
        """
        
        # Execute and get counts
        simulation_results = execute(qobj, self.aer_simulator).result()

        job = OpenSuperQ_Job(None)
            
        config = self.configuration()
        job._result = Result.from_dict({
            'results':         simulation_results,
            'backend_name':    config.backend_name,
            'backend_version': config.backend_version,
            'qobj_id':         '0',
            'job_id':          '0',
            'success':         True
        })
        
        return job
    

class OpenSuperQ_Provider(BaseProvider):
    """Provider for the Hadamard backend"""

    def __init__(self, *args, **kwargs):
        super().__init__(args, kwargs)

        # Populate the list of Hadamard backends
        self._backends = [ETH_7_rev_1_Backend(provider=self)]

    def backends(self, name=None, filters=None, **kwargs):
        # pylint: disable=arguments-differ
        backends = self._backends
        if name:
            backends = [backend for backend in backends if backend.name() == name]

        return filter_backends(backends, filters=filters, **kwargs)


qreg = QuantumRegister(4)
creg = ClassicalRegister(4)
qc = QuantumCircuit(qreg, creg)
qc.h(qreg[3])
qc.h(qreg[1])
qc.h(qreg[3])
qc.h(qreg[2])
qc.measure(qreg, creg)


provider = OpenSuperQ_Provider()

job = execute(qc, provider.get_backend('OSQ_ETH7_rev1'), 
                           pass_manager=PassManager(), shots=1024)
results = job.result()

print(results.get_counts(qc))


ModelValidationError: {'gates': {0: {}, '_schema': ['Invalid input type.', 'Invalid input type.', 'Invalid input type.', 'Invalid input type.', 'Invalid input type.'], 1: {}, 2: {}, 3: {}, 4: {}}}

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table